<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/fish_speech_tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/fishaudio/fish-speech.git

In [47]:
%cd /content/fish-speech

/content/fish-speech


In [ ]:
!apt-get install portaudio19-dev

In [ ]:
!pip install -e .[stable]

In [ ]:
!huggingface-cli download fishaudio/fish-speech-1.5 --local-dir checkpoints/fish-speech-1.5/

# infer

## 1. Encode reference audio: / 从语音生成 prompt:

In [ ]:
!wget https://github.com/FunAudioLLM/CosyVoice/raw/refs/heads/main/asset/zero_shot_prompt.wav -O zero_shot_prompt.wav

In [10]:
from IPython.display import Audio
Audio("zero_shot_prompt.wav",autoplay=True)

In [46]:
# VQ Encoder -> numpy indices
!python fish_speech/models/vqgan/inference.py \
    -i zero_shot_prompt.wav \
    --checkpoint-path "checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth"

/usr/local/lib/python3.11/dist-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:445: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/usr/local/lib/python3.11/dist-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:630: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/usr/local/lib/python3.11/dist-packages/vector_quantize_pytorch/finite_scalar_quantization.py:147: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/usr/local/lib/python3.11/dist-packages/vector_quantize_pytorch/lookup_free_quantization.py:209: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
2025

In [12]:
from IPython.display import Audio
Audio("fake.wav",autoplay=True)

In [14]:
!ls -hl fake.npy fake.wav

-rw-r--r-- 1 root root 2.5K Jan 21 10:23 fake.npy
-rw-r--r-- 1 root root 301K Jan 21 10:23 fake.wav


## 2. Generate semantic tokens from text: / 从文本生成语义 token:

- 该命令会在工作目录下创建 codes_N 文件, 其中 N 是从 0 开始的整数.
- 可以使用 `--compile` 来融合 cuda 内核以实现更快的推理 (~30 tokens/秒 -> ~300 tokens/秒)

In [40]:
!python fish_speech/models/text2semantic/inference.py \
    --text "hello world,你叫什么名字，能讲一个故事吗？" \
    --checkpoint-path "checkpoints/fish-speech-1.5" \
    --num-samples 2 \
    #--compile

2025-01-21 11:27:56.444 | INFO     | __main__:main:1053 - Loading model ...
2025-01-21 11:28:08.305 | INFO     | __main__:load_model:681 - Restored model from checkpoint
2025-01-21 11:28:08.306 | INFO     | __main__:load_model:687 - Using DualARTransformer
2025-01-21 11:28:08.317 | INFO     | __main__:main:1067 - Time to load model: 11.87 seconds
<|im_start|>user
hello world,你叫什么名字，能讲一个故事吗？<|im_end|><|im_start|>assistant
<|voice|>
2025-01-21 11:28:08.338 | INFO     | __main__:generate_long:788 - Encoded text: hello world,你叫什么名字，能讲一个故事吗？
2025-01-21 11:28:08.339 | INFO     | __main__:generate_long:806 - Generating sentence 1/1 of sample 1/2
  0% 0/8160 [00:00<?, ?it/s]/usr/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
  1% 81/8160 [00:07<12:26, 10.82it/s

In [41]:
!python fish_speech/models/text2semantic/inference.py \
    --text "hello world,你叫什么名字，能讲一个故事吗？" \
    --prompt-text "" \
    --prompt-tokens "fake.npy" \
    --checkpoint-path "checkpoints/fish-speech-1.5" \
    --num-samples 2
    # --compile

2025-01-21 11:28:36.058 | INFO     | __main__:main:1053 - Loading model ...
2025-01-21 11:28:47.851 | INFO     | __main__:load_model:681 - Restored model from checkpoint
2025-01-21 11:28:47.852 | INFO     | __main__:load_model:687 - Using DualARTransformer
2025-01-21 11:28:47.863 | INFO     | __main__:main:1067 - Time to load model: 11.81 seconds
<|im_start|>user
<|im_end|><|im_start|>assistant
<|voice|><|semantic:528|><|semantic:360|><|semantic:361|><|semantic:360|><|semantic:368|><|semantic:360|><|semantic:104|><|semantic:40|><|semantic:641|><|semantic:800|><|semantic:920|><|semantic:532|><|semantic:449|><|semantic:20|><|semantic:848|><|semantic:761|><|semantic:327|><|semantic:422|><|semantic:745|><|semantic:456|><|semantic:880|><|semantic:729|><|semantic:489|><|semantic:722|><|semantic:512|><|semantic:297|><|semantic:329|><|semantic:723|><|semantic:612|><|semantic:661|><|semantic:809|><|semantic:538|><|semantic:620|><|semantic:502|><|semantic:348|><|semantic:465|><|semantic:731|><|s

In [45]:
!python fish_speech/models/text2semantic/inference.py \
    --text "hello world,你叫什么名字，能讲一个故事吗？" \
    --prompt-text "开心" \
    --prompt-tokens "fake.npy" \
    --checkpoint-path "checkpoints/fish-speech-1.5" \
    --num-samples 1


2025-01-21 12:10:58.760 | INFO     | __main__:main:1053 - Loading model ...
2025-01-21 12:11:10.185 | INFO     | __main__:load_model:681 - Restored model from checkpoint
2025-01-21 12:11:10.185 | INFO     | __main__:load_model:687 - Using DualARTransformer
DualARTransformer(
  (embeddings): Embedding(102048, 1024)
  (codebook_embeddings): Embedding(8192, 1024)
  (layers): ModuleList(
    (0-23): 24 x TransformerBlock(
      (attention): Attention(
        (wqkv): Linear(in_features=1024, out_features=1280, bias=False)
        (wo): Linear(in_features=1024, out_features=1024, bias=False)
      )
      (feed_forward): FeedForward(
        (w1): Linear(in_features=1024, out_features=4096, bias=False)
        (w3): Linear(in_features=1024, out_features=4096, bias=False)
        (w2): Linear(in_features=4096, out_features=1024, bias=False)
      )
      (ffn_norm): RMSNorm()
      (attention_norm): RMSNorm()
    )
  )
  (norm): RMSNorm()
  (output): Linear(in_features=1024, out_features=102

### 3. Generate speech from semantic tokens: / 从语义 token 生成人声:

In [43]:
# # codebook VQ Decoder -> waveform
!python fish_speech/models/vqgan/inference.py \
    -i "codes_0.npy" \
    --checkpoint-path "checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth"

/usr/local/lib/python3.11/dist-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:445: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/usr/local/lib/python3.11/dist-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:630: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/usr/local/lib/python3.11/dist-packages/vector_quantize_pytorch/finite_scalar_quantization.py:147: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/usr/local/lib/python3.11/dist-packages/vector_quantize_pytorch/lookup_free_quantization.py:209: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
2025

In [44]:
Audio("fake.wav",autoplay=True)

In [36]:
Audio("fake.wav",autoplay=True)

# sft - LoRA

In [49]:
!cat docs/zh/finetune.md

# 微调

显然, 当你打开这个页面的时候, 你已经对预训练模型 zero-shot 的效果不算满意. 你想要微调一个模型, 使得它在你的数据集上表现更好.  

在目前版本，你只需要微调'LLAMA'部分即可.

## LLAMA 微调
### 1. 准备数据集

```
.
├── SPK1
│   ├── 21.15-26.44.lab
│   ├── 21.15-26.44.mp3
│   ├── 27.51-29.98.lab
│   ├── 27.51-29.98.mp3
│   ├── 30.1-32.71.lab
│   └── 30.1-32.71.mp3
└── SPK2
    ├── 38.79-40.85.lab
    └── 38.79-40.85.mp3
```

你需要将数据集转为以上格式, 并放到 `data` 下, 音频后缀可以为 `.mp3`, `.wav` 或 `.flac`, 标注文件后缀建议为 `.lab`.

!!! info
    标注文件 `.lab` 仅需包含音频的转写文本，无需遵循特殊格式要求。例如，如果 `hi.mp3` 中的内容是“你好，再见。”，那么 `hi.lab` 文件中只需包含一行文本：“你好，再见”。    

!!! warning
    建议先对数据集进行响度匹配, 你可以使用 [fish-audio-preprocess](https://github.com/fishaudio/audio-preprocess) 来完成这一步骤. 
    ```bash
    fap loudness-norm data-raw data --clean
    ```

### 2. 批量提取语义 token

确保你已经下载了 vqgan 权重, 如果没有, 请运行以下命令:

```bash
huggingface-cli download fishaudio/fish-speech-1.5 --local-dir checkpoints/fish-speech-1.5
```

对于中国大陆用户, 可使用 mirror 下载.

```bash
HF_ENDPOINT=https://hf-mirror.com huggingface-cli download fishaudio/fish